In [1]:
import pandas as pd                                       #importing pandas
import numpy as np                                        #importing numpy
import matplotlib.pyplot as plt                           #importing matplotlib 
import seaborn as sns 
from gurobipy import Model, GRB, quicksum

### Basic integer knapsack 🎒
* maximize value subject to weight constraint and volume constraint
* only one of each kind of item

In [2]:
weights = [5,20,90,11,17,22,56,74,38,1,17,8,42,44,15]
values = [1,10,4,19,2,6,3,9,13,0,2,1,8,5,4]
volumes = [0.5,1,0.25,2,5,3,0.5,0.1,0.5,1,0.3,0.9,0.25,3,5]
assert len(weights) == len(values) == len(volumes)

In [3]:
indices = range(len(weights))
vals = dict(zip(indices, values))
weight = dict(zip(indices, weights))
vols = dict(zip(indices, volumes))
weight_cap = 130
vol_cap = 10

m = Model();

Restricted license - for non-production use only - expires 2023-10-25


In [4]:
decision_var = m.addVars(indices, vtype=GRB.BINARY, name="decision_var")
m.setObjective(quicksum(vals[i]*decision_var[i] for i in indices), GRB.MAXIMIZE)
m.addConstr(quicksum(weight[i]*decision_var[i] for i in indices) <= weight_cap, name="weight");
m.addConstr(quicksum(vols[i]*decision_var[i] for i in indices) <= vol_cap, name="weight");

In [5]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 15 columns and 30 nonzeros
Model fingerprint: 0x191d54e3
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e-01, 9e+01]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+02]
Found heuristic solution: objective 34.0000000
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 2 rows, 13 columns, 26 nonzeros
Found heuristic solution: objective 49.0000000
Variable types: 0 continuous, 13 integer (13 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 5.618102e+01, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   56.18102    0 

In [6]:
results = pd.DataFrame({'weights': weights, 'volumes': volumes, 'values': values, 'selected': False})
selected = []
for v in m.getVars():
    if v.x > 1e-6:
        selected.append(v.index)

results.loc[selected,'selected'] = True
print('Total value: ', m.objVal)
results

Total value:  54.0


,weights,volumes,values,selected
0,5,0.50,1,False
1,20,1.00,10,True
2,90,0.25,4,False
3,11,2.00,19,True
4,17,5.00,2,False
5,22,3.00,6,False
6,56,0.50,3,False
7,74,0.10,9,False
8,38,0.50,13,True
9,1,1.00,0,False


In [7]:
results[results['selected']].sum()

weights     126.00
volumes       8.75
values       54.00
selected      5.00
dtype: float64